In [16]:
# !pip install langchainhub -q

In [15]:
import sys
import os

from google.cloud import aiplatform, storage
from langchain import hub
from llama_index.core import (
    Document,
    PromptTemplate,
    Settings,
    SimpleDirectoryReader,
    StorageContext,
    SummaryIndex,
    VectorStoreIndex,
)
from llama_index.core.agent import ReActAgent
from llama_index.core.base.base_query_engine import BaseQueryEngine
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.objects import ObjectIndex
from llama_index.core.prompts import LangchainPromptTemplate
from llama_index.core.prompts.base import BasePromptTemplate
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.embeddings.vertex import VertexTextEmbedding
from llama_index.llms.vertex import Vertex
from llama_index.vector_stores.vertexaivectorsearch import VertexAIVectorStore

In [30]:
GCP_PROJECTS = !gcloud config get-value project
PROJECT_ID = GCP_PROJECTS[0]
REGION = "us-central1" 
GCS_BUCKET = "vertex_ai_rag_bucket"
VS_INDEX_NAME = "llamaindex_doc_index"
VS_INDEX_ENDPOINT_NAME = "llamaindex_doc_endpoint"
DOC_FOLDER = "./data_tutorial"
DOC_FOLDER_NEW = "./data_original"

In [18]:
!mkdir {DOC_FOLDER}
!gcloud storage cp gs://github-repo/generative-ai/gemini/use-cases/retrieval-augmented-generation/llamaindex/data/* {DOC_FOLDER}

print("Download completed")

Copying gs://github-repo/generative-ai/gemini/use-cases/retrieval-augmented-generation/llamaindex/data/04a01.pdf to file://./data_tutorial/04a01.pdf
Copying gs://github-repo/generative-ai/gemini/use-cases/retrieval-augmented-generation/llamaindex/data/04a02.pdf to file://./data_tutorial/04a02.pdf
Copying gs://github-repo/generative-ai/gemini/use-cases/retrieval-augmented-generation/llamaindex/data/04a03.pdf to file://./data_tutorial/04a03.pdf
Copying gs://github-repo/generative-ai/gemini/use-cases/retrieval-augmented-generation/llamaindex/data/04a04.pdf to file://./data_tutorial/04a04.pdf
  Completed files 4/4 | 3.2MiB/3.2MiB                                          

Average throughput: 35.7MiB/s
Download completed


In [19]:
sys.path.append("./scripts")

from scripts import utils

In [31]:
def setup():
    # The number of dimensions for the gecko text embeddings is 768
    VS_DIMENSIONS = 768
    
    # Vertex AI Vector Search Index configuration
    aiplatform.init(project=PROJECT_ID, location=REGION)
    
    storage_client = storage.Client()
    new_bucket = bucket = storage_client.bucket(GCS_BUCKET)
    
    # https://cloud.google.com/vertex-ai/docs/vector-search/overview
    vs_index = utils.create_vector_search_index(VS_INDEX_NAME, VS_DIMENSIONS)
    vs_endpoint = utils.create_vector_search_endpoint(VS_INDEX_ENDPOINT_NAME)
    vs_deployed_index = utils.deploy_vector_search_endpoint(
        vs_index, vs_endpoint, VS_INDEX_NAME
    )

    return new_bucket, vs_index, vs_endpoint, vs_deployed_index

In [35]:
# Create a service account for using text embedding and llm models in gcp - add to terraform

In [28]:
def initialize_llm_and_storage(
    vs_index: aiplatform.MatchingEngineIndex,
    vs_endpoint: aiplatform.MatchingEngineIndexEndpoint,
) -> StorageContext:
    """
    Initializes Vertex AI Vector Store given a Vector Search index and deployed endpoint.
    Configures embedding and LLMs models to be gecko and Gemini.
    """
    # setup storage
    vector_store = VertexAIVectorStore(
        project_id=PROJECT_ID,
        region=REGION,
        index_id=vs_index.resource_name,
        endpoint_id=vs_endpoint.resource_name,
        gcs_bucket_name=GCS_BUCKET,
    )

    # set storage context
    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    
    from google.oauth2 import service_account
    credentials = service_account.Credentials.from_service_account_file(
        "creds/my-practice-setting-c57b3dab3a57.json"
    )
    
    gemini_embedding_model = VertexTextEmbedding(
        model_name="text-embedding-005",
        project=PROJECT_ID,
        location=REGION,
        credentials=credentials,
    )
    llm = Vertex(
        model="gemini-2.0-flash",
        project=PROJECT_ID, 
        credentials=credentials
    )

    Settings.embed_model = gemini_embedding_model
    Settings.llm = llm

    return storage_context

In [32]:
(bucket, vs_index, vs_endpoint, deployed_endpoint) = setup()
storage_context = initialize_llm_and_storage(vs_index, vs_endpoint)

# Simply pass in a input directory or a list of files. It will select the best file reader based on the 
# file extensions. 
docs = SimpleDirectoryReader(DOC_FOLDER).load_data()

Creating Vector Search index llamaindex_doc_index ...
Creating MatchingEngineIndex
Create MatchingEngineIndex backing LRO: projects/535465771333/locations/us-central1/indexes/125698368310607872/operations/5358837945123995648
MatchingEngineIndex created. Resource name: projects/535465771333/locations/us-central1/indexes/125698368310607872
To use this MatchingEngineIndex in another session:
index = aiplatform.MatchingEngineIndex('projects/535465771333/locations/us-central1/indexes/125698368310607872')
Vector Search index llamaindex_doc_index created with resource name projects/535465771333/locations/us-central1/indexes/125698368310607872
Creating Vector Search index endpoint llamaindex_doc_endpoint ...
Creating MatchingEngineIndexEndpoint
Create MatchingEngineIndexEndpoint backing LRO: projects/535465771333/locations/us-central1/indexEndpoints/5336662204341026816/operations/8884874978378383360
MatchingEngineIndexEndpoint created. Resource name: projects/535465771333/locations/us-central1

ValueError: Either provide credentials or all of client_email, token_uri, private_key_id, and private_key.